In [12]:
import networkx as nx
import numpy as np
from scipy.special import comb
from itertools import combinations 
import networkx.algorithms.isomorphism as iso
from tqdm import tqdm
import matplotlib.pyplot as plt
import dgl

In [18]:
def generate_graphlet(n):
    non_iso_graph = []
    non_iso_graph_adj = []
    dgl_graph = []
    for i in tqdm(range(n-1, int(comb(n, 2))+1)):
    # for each of these possible # of edges
        arr = np.array(range(int((n**2-n)/2)))
        all_comb = list(combinations(arr, i)) 
        # all possible combination of edge positions 
        indices = np.triu_indices(n, 1)
        for m in range(len(all_comb)):
            # iterate over all these graphs
            adj = np.zeros((n,n))
            adj[indices[0][np.array(all_comb[m])], indices[1][np.array(all_comb[m])]] = 1
            adj_temp = adj
            adj = adj + adj.T
            #print(adj)
            if sum(np.sum(adj_temp, axis = 0) == 0) == 1:
                #the graph has to be connected
                new_graph = nx.from_numpy_matrix(adj)
                if len(non_iso_graph) == 0:
                    non_iso_graph.append(new_graph)
                    non_iso_graph_adj.append(adj)
                    S = dgl.DGLGraph()
                    S.from_networkx(new_graph)
                    dgl_graph.append(S)
                else:
                    is_iso = False
                    for g in non_iso_graph:
                        if iso.is_isomorphic(g, new_graph):
                            #print('yes')
                            is_iso = True
                            break
                    if not is_iso:
                        # not isomorphic to any of the current graphs
                        non_iso_graph.append(new_graph)
                        non_iso_graph_adj.append(adj)
                        
                        S = dgl.DGLGraph()
                        S.from_networkx(new_graph)
                        dgl_graph.append(S)
                        
    
    print('There are {} non-isomorphic graphs'.format(len(non_iso_graph)))
    return dgl_graph
#for i in range(len(non_iso_graph)):
#    plt.figure(i+1)
#    nx.draw(non_iso_graph[i], with_labels = True)

In [19]:
graphs = []
for i in range(1, 5):
    graphs = graphs + generate_graphlet(i+1)

100%|██████████| 7/7 [00:00<00:00, 34.96it/s]

There are 1 non-isomorphic graphs
There are 2 non-isomorphic graphs
There are 6 non-isomorphic graphs
There are 21 non-isomorphic graphs


In [20]:
len(graphs)

30

In [22]:
(dgl.batch(graphs))

137